In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from urllib.parse import urlparse
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding, LSTM, Bidirectional
import re
import ipaddress
from bs4 import BeautifulSoup
import requests
from keras import callbacks
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')
from flask import Flask, render_template, request

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok

In [ ]:
from flask_ngrok import run_with_ngrok

In [ ]:
!ngrok authtoken 25JUqEfhazUjfb9CbxQTTgctkOz_5pMh1LJVPBWx3uxv7zWbq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import ipaddress

In [ ]:
def generate_data_set(url):
  data_set = []

  try:
    ipaddress.ip_address(url)
    data_set.append(1)
  except:
    data_set.append(0)

  if "@" in url:
    data_set.append(1)
  else:
    data_set.append(0)

  if len(url) < 54:
    data_set.append(0)
  else:
    data_set.append(1)

  protocol = urlparse(url)
  if protocol.scheme == 'https':
    data_set.append(1)
  else:
    data_set.append(0)

  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  data_set.append(depth)

  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      data_set.append(1)
    else:
      data_set.append(0)
  else:
    data_set.append(0)

  if '-' in urlparse(url).netloc:
    data_set.append(1)
  else:
    data_set.append(0)


  shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                        r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                        r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                        r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                        r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                        r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                        r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                        r"tr\.im|link\.zip\.net"

  match=re.search(shortening_services,url)
  if match:
    data_set.append(1)
  else:
    data_set.append(0)

  return data_set


In [ ]:
#Legitimate
app = Flask(__name__, template_folder='/content/templates')
run_with_ngrok(app)

model =load_model("model_bidi (1).h5")

@app.route("/", methods=['GET', 'POST'])
def index():
    return render_template("index1.html", xx=-1)


@app.route("/predict", methods=["GET", "POST"])
def predict():
    if request.method == "POST":
        url = request.form["url"]
        x = generate_data_set(url)
        x = np.array(x).reshape(1,8)
        #x = np.array(generate_data_set(url)).reshape(1,8)
        print(x)
        print(type(x))
        y_pred = (model.predict(x))[0]
        print(y_pred)

        # y_pro_phishing = model.predict_proba(x)[0,0]
        # y_pro_non_phishing = model.predict_proba(x)[0,1]
        y_pro_phishing = x[0, 0]
        y_pro_non_phishing = x[0, 1]
        if y_pred > 0.5:
            pred = y_pro_phishing * 100
            return render_template('index1.html', prediction_text='It is {0:.2f} % safe to go'.format(pred))
        else:
            pred = y_pro_non_phishing * 100
            return render_template('index1.html', prediction_text='It is {0:.2f} % unsafe to go '.format(pred))
    return render_template("index1.html", xx=-1)


if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://df0d-35-236-231-51.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Mar/2022 05:27:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 05:27:37] "GET /favicon.ico HTTP/1.1" 404 -


[[0 0 0 1 0 0 0 0]]
<class 'numpy.ndarray'>


127.0.0.1 - - [26/Mar/2022 05:27:41] "POST /predict HTTP/1.1" 200 -


[4.6858634e-07]


In [ ]:
#Phishing
app = Flask(__name__, template_folder='/content/templates')
run_with_ngrok(app)

model =load_model("model_bidi (1).h5")

@app.route("/", methods=['GET', 'POST'])
def index():
    return render_template("index.html", xx=-1)


@app.route("/predict", methods=["GET", "POST"])
def predict():
    if request.method == "POST":
        url = request.form["url"]
        x = generate_data_set(url)
        x = np.array(x).reshape(1,8)
        #x = np.array(generate_data_set(url)).reshape(1,8)
        print(x)
        print(type(x))
        y_pred = (model.predict(x))[0]
        print(y_pred)

        # y_pro_phishing = model.predict_proba(x)[0,0]
        # y_pro_non_phishing = model.predict_proba(x)[0,1]
        y_pro_phishing = x[0, 0]
        y_pro_non_phishing = x[0, 1]
        if y_pred > 0.5:
            pred = y_pro_phishing * 100
            return render_template('index.html', prediction_text='It is {0:.2f} % safe to go'.format(pred))
        else:
            pred = y_pro_non_phishing * 100
            return render_template('index.html', prediction_text='It is {0:.2f} % unsafe to go '.format(pred))
    return render_template("index.html", xx=-1)


if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2350-35-236-231-51.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Mar/2022 05:28:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 05:28:04] "GET /favicon.ico HTTP/1.1" 404 -


[[0 0 1 1 3 0 0 0]]
<class 'numpy.ndarray'>


127.0.0.1 - - [26/Mar/2022 05:28:15] "POST /predict HTTP/1.1" 200 -


[0.02856791]
